# Model Versioning for Keras with VersionTreeNode

In [1]:
import numpy as np
import pickle

from grid.ipfsapi.version_tree import VersionTreeNode
from grid.ipfsapi.client import Client as IpfsClient

/Users/yanndupis/anaconda2/envs/openmined2/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
client = IpfsClient()

This notebook illustrate version control of the weights with VersionTreeNode api.
- First we split the MNIST data in three parts.
- We train the model on each part. Every time we train the model on a subset of the original dataset, we commit the weights.
- VersionTreeNode gives us the abilty to get access to the historical version of the weights by listing ancestors.

Load MNIST data.

In [3]:
# The original model can be found here: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


Split the data.

In [4]:
size_part = len(x_train)//3
x_train_part_1 = x_train[:size_part,: :]
y_train_part_1 = y_train[:size_part]

x_train_part_2 = x_train[size_part:size_part*2,: :] 
y_train_part_2 = y_train[size_part:size_part*2]

x_train_part_3 = x_train[size_part*2:size_part*3,: :]
y_train_part_3 =  y_train[size_part*2:size_part*3]

Create the model.

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

#### Fit model on the first part then commit weights

In [6]:
model.fit(x_train_part_1, y_train_part_1,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 20000 samples, validate on 10000 samples
Epoch 1/1
20000/20000 [==============================] - 101s 5ms/step - loss: 0.5134 - acc: 0.8410 - val_loss: 0.1372 - val_acc: 0.9573
Test loss: 0.13722231417149305
Test accuracy: 0.9573


Commit weigths after training the model on the first part of the MNIST data.

In [7]:
# Get the weights of the model
weights_part_1 = model.get_weights()
# Create a node to store the weights
n1 = VersionTreeNode(pickle.dumps(weights_part_1), None)
# commit the node to write the weights to IPFS. 
# Set parent_hash to None if the first version of weights (root of the tree)
n1_ipfs_address = n1.commit(client)
print(n1_ipfs_address)

model.set_weights(pickle.loads(n1.contents))

QmT6Wjd4V2tZXiyxhkAFhB4srApLRi3T5ycYEk6GzonJP6


#### Fit model on the second part then commit weights

In [8]:
model.fit(x_train_part_2, y_train_part_2,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 20000 samples, validate on 10000 samples
Epoch 1/1
20000/20000 [==============================] - 92s 5ms/step - loss: 0.1625 - acc: 0.9508 - val_loss: 0.0681 - val_acc: 0.9790
Test loss: 0.06812974594687111
Test accuracy: 0.979


In [9]:
weights_part_2 = model.get_weights()

n2 = VersionTreeNode(pickle.dumps(weights_part_2), n1_ipfs_address)

n2_ipfs_address = n2.commit(client)
print(n2_ipfs_address)

model.set_weights(pickle.loads(n2.contents))

QmbdXamahfz2r93kTKhmQkpsT6WNTjqT2ZmcCgHuy25KbW


#### Fit model on the third part then commit weights

In [10]:
model.fit(x_train_part_3, y_train_part_3,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 20000 samples, validate on 10000 samples
Epoch 1/1
20000/20000 [==============================] - 78s 4ms/step - loss: 0.1267 - acc: 0.9633 - val_loss: 0.0632 - val_acc: 0.9801
Test loss: 0.0631736337831011
Test accuracy: 0.9801


In [11]:
weights_part_3 = model.get_weights()

n3 = VersionTreeNode(pickle.dumps(weights_part_3), parent_hash=n2_ipfs_address)

n3_ipfs_address = n3.commit(client)

#### Get historical weights

In [12]:
node_ancestors = n3.get_with_ancestors(client)

In [13]:
historical_weights = [pickle.loads(x.contents) for x in node_ancestors]

In [14]:
for i in range(len(historical_weights)):
    model.set_weights(historical_weights[i])
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

Test loss: 0.0631736337831011
Test accuracy: 0.9801
Test loss: 0.06812974594687111
Test accuracy: 0.979
Test loss: 0.13722231417149305
Test accuracy: 0.9573


The test loss and accuracy are matching the original loss and accuracy before commit.